# Sarah Gets a Diamond - Starter Code

In this code, we will build a model to predict diamond prices based on the features of the diamond. We will then compete as a class as to who can build the most predictive model.

## Importing

Like all of our starter code, we will start by loading a set of "modules" that contain useful functionality for the assignment. We will use most of these modules in every set of starter code we have, but we will customize the set for some assignment specific functionality. The way this works is we specify the name of a module first, and then we can either import it with a shorthand phrase of import certain functions.

For example, the code `import numpy as np` says to import the numpy module (a module for handling large datasets as arrays that is very common in data science) and call it `np`. Then if we ever want to use a function from that module, for example the `array` function, we would type `np.array(x)` (where `x` is the data we are giving to the function).

Alternatively, we can import specific functions directly. Below we write `from scipy.optimize import minimize_scalar`, which says to take "from" the `scipy.optimize` module the function `minimize_scalar`. This allows us to use the function `minimize_scalar` just by writing `minimize_scalar`.

We can also import all functions from a module at a time like we do with `from math import *`. This imports all of the functions from math, which includes things like `log()` to calculate the natural logarithm of a number.

In [1]:
import numpy as np
import pandas as pd
from math import *
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib as mpl

In [2]:
# This is a comment. Anything in a "code cell" that is preceeded by a "#" is a comment
# and it will not be interpreted as code to be run when you run the cell.
# This sets some nicer defaults for plotting.
# This must be run in a separate cell from importing matplotlib due to a bug.
params = {'legend.fontsize': 'large',
          'figure.figsize': (11.0, 11.0),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'xx-large',
          'xtick.labelsize':'large',
          'ytick.labelsize':'large'}
mpl.rcParams.update(params)

# This makes it so that the pandas dataframes don't get truncated horizontally.
pd.options.display.max_columns = 200

If you are uncertain what a function does, you can look up help in the jupyter notebook by writing `?np.array` (replace `np.array` with the relevant function). The next cell gives us the documentation for the `pandas` function, `read_csv`. Since we imported the `pandas` library as `pd` above, we reference that function by `pd.read_csv`.

In [3]:
?pd.read_csv

Signature:
pd.read_csv(
    filepath_or_buffer: 'FilePathOrBuffer',
    sep=<no_default>,
    delimiter=None,
    header='infer',
    names=<no_default>,
    index_col=None,
    usecols=None,
    squeeze=False,
    prefix=<no_default>,
    mangle_dupe_cols=True,
    dtype: 'DtypeArg | None' = None,
    engine=None,
    converters=None,
    true_values=None,
    false_values=None,
    skipinitialspace=False,
    skiprows=None,
    skipfooter=0,
    nrows=None,
    na_values=None,
    keep_default_na=True,
    na_filter=True,
    verbose=False,
    skip_blank_lines=True,
    parse_dates=False,
    infer_datetime_format=False,
    keep_date_col=False,
    date_parser=None,
    dayfirst=False,
    cache_dates=True,
    iterator=False,
    chunksize=None,
    compression='infer',
    thousands=None,
    decimal: 'str' = '.',
    lineterminator=None,
    quotechar='"',
    quoting=0,
    doublequote=True,
    escapechar=None,
    comment=None,
    encoding=None,
    encoding_errors: 'str | N

Sometimes the documentation that it produces helps you to figure out what you need to put in the function (you can see it gives you some examples), but sometimes, it's still quite hard to parse. In these cases, I recommend you google the function with a specific question (or an error message if you are getting an error message), and you will likely find an explanation. The website [stackoverflow.com](http://www.stackoverflow.com) is a website that you will likely become very familiar with. It is a community driven question and answering platform for coding related questions. It is also often the most useful (and top) search result that comes from googling something.

## Load and clean the data

We first have to load the data. We will read it in as a Pandas "dataframe". Dataframes are one of the most important tools for a data scientist. They store data in a structured format, kind of like an excel spreadsheet.

We can reference the data set with "train.csv" and the test set with "test.csv" because our Juptyer Notebook is in the same folder as the data set. If the notebook was in a different folder, we would have to provide a longer path to the file. If you ever create your own notebook and try to read in a file (or you upload new data to use in a Jupyter Notebook), you will either have to put the notebook and the data set in the same folder, or you will have to give it a more complete path to the data set. We will see examples of this later in the course.

We store the training data in a "variable" called `df_train` and the testing data in a variable called `df_test`, and anytime we want to reference the data, we will use that variable. This convention will be fairly common for us.

In [4]:
df_train = pd.read_csv("train.csv", index_col="ID")
df_test = pd.read_csv("test.csv", index_col="ID")


Notice that when we called the function `pd.read_csv`, we gave it (or _called_ it with) two _arguments_. The first argument is called a positional argument (i.e., the function knows what it is based on the position, namely that it is first, in the function call). This argument is the name of the dataset to read in.

The second argument is called a _keyword_ argument because we called the function with a keyword set equal to something. In this case, we used the keyword `index_col` and we set it equal to the value `"ID"`. You can look back up at the function documentation from above (the `?pd.read_csv` cell) to see what the argument does.

We can find the part of the documentation and read what that argument does, but I've included it below for easier reading (but it is just copy and pasted from above).

```
index_col : int, str, sequence of int / str, or False, default ``None``
  Column(s) to use as the row labels of the ``DataFrame``, either given as
  string name or column index. If a sequence of int / str is given, a
  MultiIndex is used.

  Note: ``index_col=False`` can be used to force pandas to *not* use the first
  column as the index, e.g. when you have a malformed file with delimiters at
  the end of each line.
```

Looking at the documentation, we can see that `index_col` gives each row of our data a name, and specifically the name it uses comes from the column named "ID" in the original csv file.

Let's look at the first few rows. We use a built in `method` (a function that is specific to a certain kind of variable, in this case a pandas dataframe) called `head`. This will give a view of the dataframe that should look very familiar to any excel jockey.

In [5]:
df_train.head()

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
ID,,,,,,,,
1,1.10,Ideal,H,SI1,VG,EX,GIA,"$5,169"
2,0.83,Ideal,H,VS1,ID,ID,AGSL,"$3,470"
3,0.85,Ideal,H,SI1,EX,EX,GIA,"$3,183"
4,0.91,Ideal,E,SI1,VG,VG,GIA,"$4,370"
5,0.83,Ideal,G,SI1,EX,EX,GIA,"$3,171"


What does `head` do?

In [6]:
?df_train.head

Signature: df_train.head(n: 'int' = 5) -> 'FrameOrSeries'
Docstring:
Return the first `n` rows.

This function returns the first `n` rows for the object based
on position. It is useful for quickly testing if your object
has the right type of data in it.

For negative values of `n`, this function returns all rows except
the last `n` rows, equivalent to ``df[:-n]``.

Parameters
----------
n : int, default 5
    Number of rows to select.

Returns
-------
same type as caller
    The first `n` rows of the caller object.

See Also
--------
DataFrame.tail: Returns the last `n` rows.

Examples
--------
>>> df = pd.DataFrame({'animal': ['alligator', 'bee', 'falcon', 'lion',
...                    'monkey', 'parrot', 'shark', 'whale', 'zebra']})
>>> df
      animal
0  alligator
1        bee
2     falcon
3       lion
4     monkey
5     parrot
6      shark
7      whale
8      zebra

Viewing the first 5 lines

>>> df.head()
      animal
0  alligator
1        bee
2     falcon
3       lion
4     monk

Note that we didn't give the function `df_train.head()` any arguments, but we could have. It takes up to one argument called `n`, so we could have also called the function like as follows.

In [7]:
df_test.head(10)

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report
ID,,,,,,,
6001,2.18,Very Good,I,SI1,EX,EX,GIA
6002,2.32,Ideal,E,VVS2,VG,EX,GIA
6003,0.77,Good,F,VS1,VG,G,GIA
6004,2.01,Very Good,G,VS2,EX,EX,GIA
6005,1.39,Very Good,E,VVS2,VG,EX,GIA
6006,1.22,Ideal,H,VS1,ID,ID,AGSL
6007,1.01,Very Good,E,VS1,G,VG,GIA
6008,1.50,Ideal,F,SI1,VG,VG,GIA
6009,0.90,Very Good,F,SI1,EX,EX,GIA


The function is only given one argument, and since there is only one position, it knows the argument is the `n` argument. We could also give it the name (or keyword) of the argument as well.

In [8]:
df_test.head(n=12)

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report
ID,,,,,,,
6001,2.18,Very Good,I,SI1,EX,EX,GIA
6002,2.32,Ideal,E,VVS2,VG,EX,GIA
6003,0.77,Good,F,VS1,VG,G,GIA
6004,2.01,Very Good,G,VS2,EX,EX,GIA
6005,1.39,Very Good,E,VVS2,VG,EX,GIA
6006,1.22,Ideal,H,VS1,ID,ID,AGSL
6007,1.01,Very Good,E,VS1,G,VG,GIA
6008,1.50,Ideal,F,SI1,VG,VG,GIA
6009,0.90,Very Good,F,SI1,EX,EX,GIA


We can see that `n` controls how many rows we get back. We can also see that the index (`ID`) of the `df_test` starts at `6001` instead of `1` because the ID of the rows in our test set start at `6001`.

How big are our dataset? We can use another variable specific value called `shape`. Now this is a property (something that just is for the dataframe), not a function. So, we don't put `()` after it.

In [9]:
df_train.shape

(6000, 8)

In [10]:
df_test.shape

(3142, 7)

What does it actually tell us? Well, the first number is the number of rows, and the second is the number of columns. So, in our training set we have 6000 diamonds in our data set, and 8 pieces of data about each diamond. Similarly, for the test data set, we have 3142 diamonds and seven pieces of data. Note that we have one fewer pieces of data in our testing set. Think about why for a moment... It's because we don't have prices in the testing set, we will be predicting on the testing set and submitting our predictions for the competition.

Lett's take a closer look at our dataframe to see what is actually in our data. Below, I define a "function" that takes in a dataframe and spits out nice summary of a bunch of different factors for each column in the dataframe. We will use this function in nearly every assignment we have, but we don't have to rewrite it. We can just copy and paste it to our new code. Also, we only have to define it once. Once it has been defined, we can use it in the rest of our code. Let's use it to get a better understanding of this particular dataframe.

In [11]:
def summarize_dataframe(df):
    """Summarize a dataframe, and report missing values."""
    missing_values = pd.concat([pd.DataFrame(df.columns, columns=['Variable Name']), 
                      pd.DataFrame(df.dtypes.values.reshape([-1,1]), columns=['Data Type']),
                      pd.DataFrame(df.isnull().sum().values, columns=['Missing Values']), 
                      pd.DataFrame([df[name].nunique() for name in df.columns], columns=['Unique Values'])], 
                     axis=1).set_index('Variable Name')
    with pd.option_context("display.max_rows", 1000):
        display(pd.concat([missing_values, df.describe(include='all').transpose()], axis=1).fillna(""))

In [12]:
summarize_dataframe(df_train)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Carat Weight,float64,0,196,6000.0,,,,1.33452,0.475696,0.75,1.0,1.13,1.59,2.91
Cut,object,0,5,6000.0,5,Ideal,2482,,,,,,,
Color,object,0,6,6000.0,6,G,1501,,,,,,,
Clarity,object,0,7,6000.0,7,SI1,2059,,,,,,,
Polish,object,0,4,6000.0,4,EX,2425,,,,,,,
Symmetry,object,0,4,6000.0,4,VG,2417,,,,,,,
Report,object,0,2,6000.0,2,GIA,5266,,,,,,,
Price,object,0,4821,6000.0,4821,"$4,466",8,,,,,,,


In [13]:
summarize_dataframe(df_test)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Carat Weight,float64,0,172,3142.0,,,,1.33923,0.47834,0.75,1.01,1.13,1.61,2.79
Cut,object,0,5,3142.0,5,Ideal,1301,,,,,,,
Color,object,0,6,3142.0,6,G,752,,,,,,,
Clarity,object,0,6,3142.0,6,SI1,1051,,,,,,,
Polish,object,0,4,3142.0,4,EX,1279,,,,,,,
Symmetry,object,0,4,3142.0,4,VG,1250,,,,,,,
Report,object,0,2,3142.0,2,GIA,2757,,,,,,,


We can see that in both our training and testing data, we don't have any missing data. This will greatly simplify our data preparation. We can also see the type of each data. Generally, the most common data types we'll see are `float64`, which is just a number, and `object`, which is basically anything that is not a number. The most common example of an `object` is a piece of data that is text based, and it generally means that the data is _categorical_. In this case we can see how many categories each data type has by lookin at the column "Unique Values". For example, the are five unique values for "Cut".

We can see that "Price" is also an `object` type for our training data set. This is a little unexpected because "Price" should just be a number. Let's take a look at what is in that column. We can do that by _indexing_ into our dataframe to get back just the "Price" column like below.

In [14]:
df_train['Price']

ID
1         $5,169 
2         $3,470 
3         $3,183 
4         $4,370 
5         $3,171 
          ...    
5996      $6,250 
5997      $5,328 
5998      $6,157 
5999     $11,206 
6000     $30,507 
Name: Price, Length: 6000, dtype: object

We can then immediately see what the problem is. The "Price" column includes a dollar sign "$" and a comma. That made the computer guess that the data in the column is an object. The computer doesn't know what type these pieces of data are supposed to be, so it has make it's best guess when it reads it in. It has gotten it wrong here, and we need to fix it in order to successfully build our models.

Well, the code to do that is obviously the following.

In [15]:
df_train['Price'].replace(to_replace='[\$,]', value='', regex=True).astype(float)

ID
1        5169.0
2        3470.0
3        3183.0
4        4370.0
5        3171.0
         ...   
5996     6250.0
5997     5328.0
5998     6157.0
5999    11206.0
6000    30507.0
Name: Price, Length: 6000, dtype: float64

We can see at the bottom that the data type (or `dtype`) is now `float64`, the number we were looking for.

However, the code above was in no way obvious, particularly when you are new to python.

### Aside: How would I figure that piece of code out?

As an exercise, let's walk through how one would go about figuring out both how to convert a column of data that includes dollar signs and commas into a numerical value. This is not something that you should expect to know off the top of your head or memorize. You should instead learn how to re-discover this kind of step for yourself when necessary.

The first thing I did was google "convert dollar value to numeric pandas." The first result was the following [Stack overflow article](https://stackoverflow.com/questions/32464280/converting-currency-with-to-numbers-in-python-pandas). Take a moment and read the question and the first answer.

The question is asking almost for what we are trying to do, but in the question there are multiple columns that they need to convert (all except for the first column). So, we need to modify their answer a little bit.

Since they are trying to convert multiple columns, they use the piece of code `df.columns[1:]` instead of the `'Price'` that was in our piece of code. Let's see what that does. Note that they're dataframe is called `df` while ours is called `df_train`, so we'll have to use `df_train.columns[1:]`.

In [16]:
df_train.columns[1:]

Index(['Cut', 'Color', 'Clarity', 'Polish', 'Symmetry', 'Report', 'Price'], dtype='object')

Ah, it gives us a list of the column names, except for the first column (which is "Carat Weight" in our data set). So, since we only want to convert a single column, then we can just type in a single column. That's why we use `df_train['Price']` instead of `df_train[df_train.columns[1:]]`.

In [17]:
df_train['Price']

ID
1         $5,169 
2         $3,470 
3         $3,183 
4         $4,370 
5         $3,171 
          ...    
5996      $6,250 
5997      $5,328 
5998      $6,157 
5999     $11,206 
6000     $30,507 
Name: Price, Length: 6000, dtype: object

The next thing that happens is the `.replace()` method. Documentation on the replace method is [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html). However, documentation can be a little hard to read, so let's just play around with it and see if we can figure out what is going on.

In [18]:
df_train['Price'].replace(to_replace='[\$,]', value='', regex=True)

ID
1         5169 
2         3470 
3         3183 
4         4370 
5         3171 
         ...   
5996      6250 
5997      5328 
5998      6157 
5999     11206 
6000     30507 
Name: Price, Length: 6000, dtype: object

Okay, if we just use the `.replace()` command with the same parameters it looks like it gives us back almost what we want. However, the `dtype` is still `object`, so we're not quite there. However, it can be unclear what part of those paramaters is necessary. Let's try taking some out and changing things to see what happens.

Below, I remove the `regex=True` parameter. Just to pull out the relevant part of the documentation:

In [19]:
df_train['Price'].replace(to_replace='[\$,]', value='')

ID
1         $5,169 
2         $3,470 
3         $3,183 
4         $4,370 
5         $3,171 
          ...    
5996      $6,250 
5997      $5,328 
5998      $6,157 
5999     $11,206 
6000     $30,507 
Name: Price, Length: 6000, dtype: object

Okay, that seemed to do nothing. Let's look at the documentation and see if we can figure out why:


```
regex: bool or same types as to_replace, default False

Whether to interpret to_replace and/or value as regular expressions. If this is True then to_replace must be a string. Alternatively, this could be a regular expression or a list, dict, or array of regular expressions in which case to_replace must be None.
```

Ah, this interprets the `to_replace` parameter as a "regular expression." A regular expression is a particular way to match characters in strings. They can get very complicated, but they are also really useful when working with string data. We won't use regular expressions much in our class, but if you are interested in reading a little more, a nice introduction can be found [here](https://www.oreilly.com/content/an-introduction-to-regular-expressions/). I don't recommend you spend much time learning regular expressions at this point in your data science journey.

Let's try to figure out just what the `to_replace` parameter does. Let's just try to replace the `$` symbol.

In [20]:
df_train['Price'].replace(to_replace='$', value='', regex=True)

ID
1         $5,169 
2         $3,470 
3         $3,183 
4         $4,370 
5         $3,171 
          ...    
5996      $6,250 
5997      $5,328 
5998      $6,157 
5999     $11,206 
6000     $30,507 
Name: Price, Length: 6000, dtype: object

That didn't work. Let's try it with the `\` like it was originally written.

In [21]:
df_train['Price'].replace(to_replace='\$', value='', regex=True)

ID
1         5,169 
2         3,470 
3         3,183 
4         4,370 
5         3,171 
          ...   
5996      6,250 
5997      5,328 
5998      6,157 
5999     11,206 
6000     30,507 
Name: Price, Length: 6000, dtype: object

Ah, that worked. So, it seems like when you write a `$` symbol in the regular expression, you have to have a `\` in front of it.

What about the comma?

In [22]:
df_train['Price'].replace(to_replace=',', value='', regex=True)

ID
1         $5169 
2         $3470 
3         $3183 
4         $4370 
5         $3171 
          ...   
5996      $6250 
5997      $5328 
5998      $6157 
5999     $11206 
6000     $30507 
Name: Price, Length: 6000, dtype: object

With the comma we don't need the `\`. Alright, something to file away for later. Let's try putting both the `\$` and the `,` in the `to_replace` parameter.

In [23]:
df_train['Price'].replace(to_replace='\$,', value='', regex=True)

ID
1         $5,169 
2         $3,470 
3         $3,183 
4         $4,370 
5         $3,171 
          ...    
5996      $6,250 
5997      $5,328 
5998      $6,157 
5999     $11,206 
6000     $30,507 
Name: Price, Length: 6000, dtype: object

Didn't do anything. So, the only thing missing from the original statement is the brackets around the `\$,`. Without the brackets, maybe we can guess that it is looking for strings like `$,` in the text to replace. Let's try that be replacing it with something we know is there like `\$5`, which is in the first row of data

In [24]:
df_train['Price'].replace(to_replace='\$5', value='', regex=True)

ID
1           ,169 
2         $3,470 
3         $3,183 
4         $4,370 
5         $3,171 
          ...    
5996      $6,250 
5997        ,328 
5998      $6,157 
5999     $11,206 
6000     $30,507 
Name: Price, Length: 6000, dtype: object

Yes, it is matching exact strings. We really want to match (and then replace) either a `$` **or** a `,`. That must be what the `[]` means. It means find either of those things and match and replace them.

In [25]:
df_train['Price'].replace(to_replace='[\$,]', value='', regex=True)

ID
1         5169 
2         3470 
3         3183 
4         4370 
5         3171 
         ...   
5996      6250 
5997      5328 
5998      6157 
5999     11206 
6000     30507 
Name: Price, Length: 6000, dtype: object

Indeed. What if we add a `5` to the things we put in the brackets?

In [26]:
df_train['Price'].replace(to_replace='[\$,5]', value='', regex=True)

ID
1          169 
2         3470 
3         3183 
4         4370 
5         3171 
         ...   
5996       620 
5997       328 
5998       617 
5999     11206 
6000      3007 
Name: Price, Length: 6000, dtype: object

We get rid of all of the `5`s. We obviously don't want that, but now we have a sense for what is going on. What if we replace `value=''` with something else?

In [27]:
df_train['Price'].replace(to_replace='[\$,]', value='*', regex=True)

ID
1         *5*169 
2         *3*470 
3         *3*183 
4         *4*370 
5         *3*171 
          ...    
5996      *6*250 
5997      *5*328 
5998      *6*157 
5999     *11*206 
6000     *30*507 
Name: Price, Length: 6000, dtype: object

Ah, `value` is what is replaced, and when we set it to `value=''`, it says replace whatever you find with nothing, which effectively gets rid of it.

The final step is just to figure out what `.astype(float)` means. Hopefully, this step is a little easier. We can see that if we don't include it, the `dtype` of the data ends up being `object`. If we do include it, it becomes `float64`. So, it's just a way of telling the computer that the data should be viewed as a number.

In [28]:
df_train['Price'].replace(to_replace='[\$,]', value='', regex=True).astype(float)

ID
1        5169.0
2        3470.0
3        3183.0
4        4370.0
5        3171.0
         ...   
5996     6250.0
5997     5328.0
5998     6157.0
5999    11206.0
6000    30507.0
Name: Price, Length: 6000, dtype: float64

We've worked through this example not because this is a very important example in and of itself, but it is very common for a programmer to find code somewhere online and have to figure out how and why it works. The process of exploring/playing with code is very helpful in figuring out what something does. Don't be afraid to add a new cell at any point in a notebook (use the `+` button in the toolbar at the top of the notebook) and experiment with changing up the code. This is the most effective way to learn.

## Prepare the data

Okay, now we actually need to add this new column of price data that is correctly labeled as a number. To do that, we create a new column called `Price_numeric` and we just say it's equal to our transformed original column. We can do that as below.

In [29]:
df_train['Price_numeric'] = df_train['Price'].replace(to_replace='[\$,]', value='', regex=True).astype(float)

Let's make sure it stuck and look at our data again.

In [30]:
summarize_dataframe(df_train)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Carat Weight,float64,0,196,6000.0,,,,1.33452,0.475696,0.75,1.0,1.13,1.59,2.91
Cut,object,0,5,6000.0,5,Ideal,2482,,,,,,,
Color,object,0,6,6000.0,6,G,1501,,,,,,,
Clarity,object,0,7,6000.0,7,SI1,2059,,,,,,,
Polish,object,0,4,6000.0,4,EX,2425,,,,,,,
Symmetry,object,0,4,6000.0,4,VG,2417,,,,,,,
Report,object,0,2,6000.0,2,GIA,5266,,,,,,,
Price,object,0,4821,6000.0,4821,"$4,466",8,,,,,,,
Price_numeric,float64,0,4821,6000.0,,,,11791.579333,10184.350051,2184.0,5150.5,7857.0,15036.5,101561.0


It worked fine.

Let's take a look a what is in one of our other columns of data. Let's say the `Cut` data. We know from the `summarize_dataframe` command that it consists of five unique values.

In [31]:
df_train['Cut']

ID
1                 Ideal
2                 Ideal
3                 Ideal
4                 Ideal
5                 Ideal
             ...       
5996              Ideal
5997          Very Good
5998              Ideal
5999    Signature-Ideal
6000              Ideal
Name: Cut, Length: 6000, dtype: object

That tells us something, but it is a bit hard to figure out what the possible values are. We can get those by using the `.value_counts()` method.

In [32]:
df_train['Cut'].value_counts()

Ideal              2482
Very Good          2428
Good                708
Signature-Ideal     253
Fair                129
Name: Cut, dtype: int64

This tells us what all of the possible types are, and it gives use the number of each type. As we can see, `Ideal` is the most common cut followed by `Very Good`.

Feel free to explore the data some more here to get familiar with what is in it. I've left a blank cell to encourge you to do that, but you can always add your own cells with the plus button on the toolbar.

## Split into `smaller_train` and `validation` Data Sets

However, we are not done yet manipulating our data. A best practice in machine learning is to further split up the training set into a smaller training set and a validation set. You can compare the performance of candidate models (each trained on the smaller training set) on the validation set. This `df_smaller_train` data set becomes our laboratory to test out different modeling decisions and try them out on the `df_validation` data set.

This way of thinking about intentionally training with less data is not intuitive if you haven't been exposed to it. Read [this](https://machinelearningmastery.com/difference-test-validation-datasets/) article to to build a better understanding of why we do this.

The following code randomly splits your training set into a smaller training set (75% of the training data) and a validation set (25% of the training data). There is no "correct" size for the smaller training set and the validation set. Here we have chosen a 75%-25% split, but we could chose something else. The tradeoff is that the more data you use for your smaller training set, the better the model, but you are less certain about it's performance due to having less validation data. The more validation data you have, the more confident you are in the model, but the less data you have to use to train. So, we will use a 75%-25% split, which is a fairly common split, but this is not a magic number.

Additionally, the split is random. I.e., it doesn't take the first 75% of the rows and use them for the smaller training set, it takes any row with a 75% chance. This is very important to avoid biases in how your data might be organized. For example, you may have your data sorted by carat weight. If you did, you would end up with your smaller training set having all of the large carat weight examples and none of the small carat weight rows.

In [33]:
df_smaller_train, df_validation = train_test_split(df_train, test_size=.25, random_state=201)

The exact command `train_test_split` is imported from `sklearn.model_selection` in our import statement. We give it the data set `df_train`. We also tell it the `test_size` which is how much of our data we want devoted to the validation set (set to `.25` or 25%).

We also include the parameter `random_state=201` in the function call. This is because the function `train_test_split` randomly splits your data. This is good practice. However, if we let it randomly split our data, everytime we run our code something different will happen (i.e. we end up with different random `df_smaller_train` and `df_validation` sets). We pass an initial _seed_ to the randomness generator in the function which makes the function give us _the same random split_ each time. This helps us with reproducibility with the code, and it is generally good practice when building models.

Let's take a look at what we have in our `df_smaller_train` data set.

In [34]:
summarize_dataframe(df_smaller_train)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Carat Weight,float64,0,187,4500.0,,,,1.327182,0.472008,0.75,1.0,1.12,1.58,2.91
Cut,object,0,5,4500.0,5,Ideal,1866,,,,,,,
Color,object,0,6,4500.0,6,G,1135,,,,,,,
Clarity,object,0,7,4500.0,7,SI1,1568,,,,,,,
Polish,object,0,4,4500.0,4,EX,1813,,,,,,,
Symmetry,object,0,4,4500.0,4,VG,1790,,,,,,,
Report,object,0,2,4500.0,2,GIA,3933,,,,,,,
Price,object,0,3773,4500.0,3773,"$4,466",6,,,,,,,
Price_numeric,float64,0,3773,4500.0,,,,11647.470667,9987.135937,2184.0,5122.0,7795.0,14703.25,96493.0


Indeed, we have `4500` rows, which is 75% of our original data.

The validation data set is 25% of the original data. 

In [35]:
summarize_dataframe(df_validation)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Carat Weight,float64,0,165,1500.0,,,,1.356533,0.486089,0.75,1.01,1.15,1.65,2.81
Cut,object,0,5,1500.0,5,Ideal,616,,,,,,,
Color,object,0,6,1500.0,6,G,366,,,,,,,
Clarity,object,0,7,1500.0,7,SI1,491,,,,,,,
Polish,object,0,4,1500.0,4,EX,612,,,,,,,
Symmetry,object,0,4,1500.0,4,VG,627,,,,,,,
Report,object,0,2,1500.0,2,GIA,1333,,,,,,,
Price,object,0,1407,1500.0,1407,"$4,771",4,,,,,,,
Price_numeric,float64,0,1407,1500.0,,,,12223.905333,10746.181569,2348.0,5227.0,7991.0,16177.0,101561.0


Now, we can use our `df_smaller_train` to build our model and our `df_validation` to confirm the performance.

## Evaluation

We need some way to determine how well our model is performing. There are lots of ways to do this. However, in this particular exercise, we are going to be using the **mean absolute error (or MAE)**. Read [this](https://www.dataquest.io/blog/understanding-regression-error-metrics/) primer on various metrics, but pay particular attention to the mean absolute error.

We will measure mean absolute error with the `mean_absolute_error()` function imported from `sklearn.metrics`. We will see how to do this after we generate some predictions.

## Advanced Regressions

Now, we fit an actual regression model. For this exercise, we are going to be using the statsmodels module. Specifically, we are going to be using a submodule called the `statsmodels.formula.api` module which we have imported as `smf`. This allows us to write down regression formulas using language that looks nearly right out of a stats textbook. We are going to walk through the regressions from the case first and see how we would do them in python.

```python
lm_1 = smf.ols(formula='Price_numeric ~ Q("Carat Weight")', data=df_smaller_train).fit()
```

We are creating a regression model and saving it in the variable `lm_1`. We are doing an ordinary least squares (or _ols_) regression, so we are using the function `smf.ols()`. We tell it the data we want to use with `data=df_smaller_train`. We define the regression model using the `formula` parameter.

The formula has the "dependent variable" on the left hand side, with the equation divided by `~`, and then the "independent variables" are on the right hand side. To reference columns of our dataframe, we just use the column names. So our dependent variable is `Price_numeric` and our independent variable is `Carat Weight`.

There is one wrinkle here. All of the variables in the formula need to be valid python variables. Unfortunately, a python variable cannot have a space in it. `Carat Weight`, however, has a space in it. So we have to wrap it up in the _quote_ function `Q()` and put some quotation marks around it to keep from getting an error.

We will be using this formula notation throughout the class, and you shouldn't expect to master it the first time you see it. As you use it and as you want to go deeper, a great resource to master this formula syntax is the [_patsy_ documentation](https://patsy.readthedocs.io/en/latest/quickstart.html). `patsy` is the package that underlies this formula syntax, and while the examples in the link are not quite what we are doing here, they are close enough to be a helpful resource.

Finally, the `.fit()` method actually trains the model on the data. Let's see it work.

### Additive Model

Let's see how we would train the additive model.

In [36]:
lm_1 = smf.ols(formula='Price_numeric ~ Q("Carat Weight")', data=df_smaller_train).fit()
lm_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Price_numeric   R-squared:                       0.738
Model:                            OLS   Adj. R-squared:                  0.738
Method:                 Least Squares   F-statistic:                 1.267e+04
Date:                Thu, 28 Oct 2021   Prob (F-statistic):               0.00
Time:                        15:54:50   Log-Likelihood:                -44811.
No. Observations:                4500   AIC:                         8.963e+04
Df Residuals:                    4498   BIC:                         8.964e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept         -1.248e+04    227.441    -54.861      0.000   -1.29e+04    -1.2e+04
Q("Carat Weight")  1.818e+04    161.466    112.579      0.000    1.79e+04    1.85e+04
==============================================================================
Omnibus:                     3668.307   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           152719.640
Skew:                           3.602   Prob(JB):                         0.00
Kurtosis:                      30.615   Cond. No.                         6.16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Above, the `.summary()` method for our regression model tells us what is in the regression model. Here we see that the intercept has a coefficient of `-1.248e+04` (this is in scientific notation, so it would 12,480 in common notation) and the `Carat Weight` has a coefficient of 18,180. This is very close to the model in the case. It's not identical because we are using a randomly selected sample of 75% of the data, so there is some variation from using the whole data set. We can also see the p values and other statistical properties.

We can now use this model to predict on our `df_validation` data. We do this with the `.predict()` method.

In [37]:
lm_1_predictions = lm_1.predict(df_validation)

Let's take a look at them.

In [38]:
lm_1_predictions

ID
1646    23877.733991
4429     7881.379434
3354     1519.192963
2608    14607.119418
3998     3882.290795
            ...     
4923     5881.835115
947      7336.049165
2152     4064.067551
4438     1519.192963
959      6245.388627
Length: 1500, dtype: float64

We can also measure our MAE for `lm_1` as follows.

In [39]:
mean_absolute_error(df_validation["Price_numeric"], lm_1_predictions)

3095.795318100367

This means that, on average, we miss the price of a diamond by about $3000. Hopefully we can do better.

### Multiplicative Model

Let's try the multiplicative model from the case. To do this, we just apply the function `np.log()`, a function that computes the natural logarithm (or _ln_ in normal math notation) to the dependent variable. It's very simple to do.

In [40]:
lm_2 = smf.ols(formula='np.log(Price_numeric) ~ Q("Carat Weight")', data=df_smaller_train).fit()
lm_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     np.log(Price_numeric)   R-squared:                       0.845
Model:                               OLS   Adj. R-squared:                  0.845
Method:                    Least Squares   F-statistic:                 2.449e+04
Date:                   Thu, 28 Oct 2021   Prob (F-statistic):               0.00
Time:                           15:54:50   Log-Likelihood:                -623.68
No. Observations:                   4500   AIC:                             1251.
Df Residuals:                       4498   BIC:                             1264.
Df Model:                              1                                         
Covariance Type:               nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             7.2674      0.012    587.551      0.000       7.243       7.292
Q("Carat Weight")     1.3743      0.009    156.505      0.000       1.357       1.391
==============================================================================
Omnibus:                      285.145   Durbin-Watson:                   2.052
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              372.276
Skew:                           0.584   Prob(JB):                     1.45e-81
Kurtosis:                       3.788   Cond. No.                         6.16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Again, we get coefficients very similar to what is observed in the case. We can also predict on our validation set.

In [41]:
lm_2_predictions = lm_2.predict(df_validation)

Let's take a look at our predictions.

In [42]:
lm_2_predictions

ID
1646    10.015898
4429     8.806545
3354     8.325552
2608     9.315023
3998     8.504206
          ...    
4923     8.655375
947      8.765317
2152     8.517949
4438     8.325552
959      8.682861
Length: 1500, dtype: float64

There seems to be a problem with our predictions. It appears like we are predicting that the price of a diamond is $8. There must be something else going on.

Remember, that we are predicting the natural logarithm of the price (`np.log(Price_numeric)`), not the price itself. So, we need to convert the predicted log prices back to true prices. We can do this with the `np.exp()` function. This is the exponential function and the exponential function "reverses" the log function.

In [43]:
np.exp(lm_2_predictions)

ID
1646    22379.432403
4429     6677.804378
3354     4128.014497
2608    11103.576378
3998     4935.485141
            ...     
4923     5740.923644
947      6408.090299
2152     5003.779981
4438     4128.014497
959      5900.903109
Length: 1500, dtype: float64

That looks a bit better. Let's see how it does for MAE.

In [44]:
mean_absolute_error(df_validation["Price_numeric"], np.exp(lm_2_predictions))

2941.669739657505

Our MAE has gone down! So, we are improving.

### Log-Log Model

Let's do the log-log model from the case. Again, this is simple. We just use the `np.log` function, but on the independent variable.

In [45]:
lm_3 = smf.ols(formula='np.log(Price_numeric) ~ np.log(Q("Carat Weight"))', data=df_smaller_train).fit()
lm_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     np.log(Price_numeric)   R-squared:                       0.866
Model:                               OLS   Adj. R-squared:                  0.866
Method:                    Least Squares   F-statistic:                 2.916e+04
Date:                   Thu, 28 Oct 2021   Prob (F-statistic):               0.00
Time:                           15:54:51   Log-Likelihood:                -287.82
No. Observations:                   4500   AIC:                             579.6
Df Residuals:                       4498   BIC:                             592.5
Df Model:                              1                                         
Covariance Type:               nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                     8.6418      0.005   1854.200      0.000       8.633       8.651
np.log(Q("Carat Weight"))     1.9893      0.012    170.766      0.000       1.966       2.012
==============================================================================
Omnibus:                      459.461   Durbin-Watson:                   2.032
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              674.271
Skew:                           0.777   Prob(JB):                    3.84e-147
Kurtosis:                       4.086   Cond. No.                         3.20
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Again, we get coefficients very similar to what is observed in the case. We can also predict on our validation set.

In [46]:
lm_3_predictions = lm_3.predict(df_validation)

Let's take a look at our predictions. Again, we need to "reverse" the log function.

In [47]:
np.exp(lm_3_predictions)

ID
1646    22486.680736
4429     7095.571053
3354     3367.196680
2608    12519.919628
3998     4592.502279
            ...     
4923     5776.608625
947      6722.488652
2152     4694.571513
4438     3367.196680
959      6006.393931
Length: 1500, dtype: float64

That looks a bit better. Let's see how it does for MAE.

In [48]:
mean_absolute_error(df_validation["Price_numeric"], np.exp(lm_3_predictions))
df_train.head()

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price,Price_numeric
ID,,,,,,,,,
1,1.10,Ideal,H,SI1,VG,EX,GIA,"$5,169",5169.0
2,0.83,Ideal,H,VS1,ID,ID,AGSL,"$3,470",3470.0
3,0.85,Ideal,H,SI1,EX,EX,GIA,"$3,183",3183.0
4,0.91,Ideal,E,SI1,VG,VG,GIA,"$4,370",4370.0
5,0.83,Ideal,G,SI1,EX,EX,GIA,"$3,171",3171.0


Our MAE has gone down again!

### Models with Multiple Independent Variables

If we want to add more variables, we can just put them in. Let's add `Cut`.

In [49]:
lm_4 = smf.ols(formula='np.log(Price_numeric) ~ Cut+ np.log(Q("Carat Weight")) + Clarity + Polish + Report', data=df_smaller_train).fit()
lm_4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     np.log(Price_numeric)   R-squared:                       0.932
Model:                               OLS   Adj. R-squared:                  0.932
Method:                    Least Squares   F-statistic:                     4103.
Date:                   Thu, 28 Oct 2021   Prob (F-statistic):               0.00
Time:                           15:54:51   Log-Likelihood:                 1235.1
No. Observations:                   4500   AIC:                            -2438.
Df Residuals:                       4484   BIC:                            -2336.
Df Model:                             15                                         
Covariance Type:               nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                     9.4307      0.110     85.955      0.000       9.216       9.646
Cut[T.Good]                   0.0245      0.021      1.167      0.243      -0.017       0.066
Cut[T.Ideal]                  0.0877      0.020      4.334      0.000       0.048       0.127
Cut[T.Signature-Ideal]        0.2713      0.025     10.933      0.000       0.223       0.320
Cut[T.Very Good]              0.0556      0.020      2.789      0.005       0.017       0.095
Clarity[T.IF]                -0.5187      0.108     -4.822      0.000      -0.730      -0.308
Clarity[T.SI1]               -1.1174      0.107    -10.468      0.000      -1.327      -0.908
Clarity[T.VS1]               -0.8843      0.107     -8.283      0.000      -1.094      -0.675
Clarity[T.VS2]               -0.9618      0.107     -9.010      0.000      -1.171      -0.753
Clarity[T.VVS1]              -0.6439      0.107     -5.999      0.000      -0.854      -0.433
Clarity[T.VVS2]              -0.7229      0.107     -6.761      0.000      -0.932      -0.513
Polish[T.G]                  -0.0481      0.011     -4.574      0.000      -0.069      -0.027
Polish[T.ID]                  0.0394      0.021      1.891      0.059      -0.001       0.080
Polish[T.VG]                 -0.0308      0.006     -4.804      0.000      -0.043      -0.018
Report[T.GIA]                 0.1202      0.018      6.531      0.000       0.084       0.156
np.log(Q("Carat Weight"))     1.9310      0.008    227.626      0.000       1.914       1.948
==============================================================================
Omnibus:                       28.365   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               36.728
Skew:                          -0.094   Prob(JB):                     1.06e-08
Kurtosis:                       3.401   Cond. No.                         168.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

If you look at the coefficients, you see that we don't have a single `Cut` variable like we have for `np.log(Q("Carat Weight"))`. Instead, we end up with a number of variables like `Cut[T.Good]`, `Cut[T.Ideal]`, `Cut[T.Signature-Ideal]`, and `Cut[T.Very Good]`. Let's figure out what that is about.

Let's look at the possible values for `Cut`. We did this above in the notebook, but let's do it again.

In [50]:
df_smaller_train['Cut'].value_counts()

Ideal              1866
Very Good          1817
Good                531
Signature-Ideal     195
Fair                 91
Name: Cut, dtype: int64

So, each of our variables corresponds to one of the possible values for `Cut`, but we don't have a variable for `Fair`. What is going on here is that the regression model `smf.ols()` knows that the `Cut` data is categorical (because the `dtype` is `object`, as we saw above), and it automatically creates dummy variables for each category. It also drops one of the categories (generally the least common class) because you are supposed to drop one dummy column to avoid overspecifciation in a linear model. So, the coefficients correspond to dummy variables for each category.

Let's see how this model performs.

In [51]:
lm_4_predictions = lm_4.predict(df_validation)

In [52]:
lm_4_predictions = lm_4.predict(df_validation)
mean_absolute_error(df_validation["Price_numeric"], np.exp(lm_4_predictions))

2051.859878979446

Even better!

### Model with Interactions

We can also easily interact two variables. Suppose that we think that `Cut` becomes more important with a higher `Carat Weight`. We can interact these two variables as follows:

In [53]:
lm_5 = smf.ols(formula='np.log(Price_numeric) ~ Cut + np.log(Q("Carat Weight")) + Cut*np.log(Q("Carat Weight"))', data=df_smaller_train).fit()
lm_5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     np.log(Price_numeric)   R-squared:                       0.876
Model:                               OLS   Adj. R-squared:                  0.876
Method:                    Least Squares   F-statistic:                     3520.
Date:                   Thu, 28 Oct 2021   Prob (F-statistic):               0.00
Time:                           15:54:51   Log-Likelihood:                -122.03
No. Observations:                   4500   AIC:                             264.1
Df Residuals:                       4490   BIC:                             328.2
Df Model:                              9                                         
Covariance Type:               nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            8.5002      0.026    321.342      0.000       8.448       8.552
Cut[T.Good]                                          0.0445      0.029      1.520      0.129      -0.013       0.102
Cut[T.Ideal]                                         0.1856      0.027      6.765      0.000       0.132       0.239
Cut[T.Signature-Ideal]                               0.3463      0.034     10.234      0.000       0.280       0.413
Cut[T.Very Good]                                     0.1148      0.027      4.194      0.000       0.061       0.168
np.log(Q("Carat Weight"))                            1.9075      0.128     14.896      0.000       1.656       2.158
Cut[T.Good]:np.log(Q("Carat Weight"))                0.0367      0.133      0.277      0.782      -0.223       0.297
Cut[T.Ideal]:np.log(Q("Carat Weight"))               0.0742      0.129      0.574      0.566      -0.179       0.327
Cut[T.Signature-Ideal]:np.log(Q("Carat Weight"))     0.1181      0.147      0.804      0.422      -0.170       0.406
Cut[T.Very Good]:np.log(Q("Carat Weight"))           0.0775      0.129      0.600      0.549      -0.176       0.331
==============================================================================
Omnibus:                      463.763   Durbin-Watson:                   2.029
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              707.260
Skew:                           0.765   Prob(JB):                    2.63e-154
Kurtosis:                       4.197   Cond. No.                         93.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The model automatically creates interactions between each category for `Cut` and `np.log(Q("Carat Weight"))`. We can then test the performance.

In [54]:
lm_5_predictions = lm_5.predict(df_validation)

In [55]:
mean_absolute_error(df_validation["Price_numeric"], np.exp(lm_5_predictions))

2706.1227231696316

That helped us, but only a little bit.

### ADVANCED: Segmenting Variables

This is an advanced section, and you are not expected to follow everything in this section. This is provided both as a resource as you progress to more advanced topics to come back to, and as a way for those who want to dive deep to go a little deeper. Feel free to skim through it.

The formula specification is very flexible. Suppose that we think the relationship between `Carat Weight` and `ln(Price_numeric)` changes for different ranges of `Carat Weight`. Suppose that we think there is one relationship that holds from weights 0 to 1, a different one that holds from 1 to 2, and a third for anything above 2. We can implement that using a linear spline as follows.

In [56]:
lm_6 = smf.ols(formula='np.log(Price_numeric) ~ Q("Carat Weight") + np.maximum(Q("Carat Weight") - 1, 0) + np.maximum(Q("Carat Weight") - 2, 0)', data=df_smaller_train).fit()
lm_6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     np.log(Price_numeric)   R-squared:                       0.870
Model:                               OLS   Adj. R-squared:                  0.869
Method:                    Least Squares   F-statistic:                     9991.
Date:                   Thu, 28 Oct 2021   Prob (F-statistic):               0.00
Time:                           15:54:52   Log-Likelihood:                -233.30
No. Observations:                   4500   AIC:                             474.6
Df Residuals:                       4496   BIC:                             500.2
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                                5.7503      0.061     94.597      0.000       5.631       5.870
Q("Carat Weight")                        2.9668      0.064     46.459      0.000       2.842       3.092
np.maximum(Q("Carat Weight") - 1, 0)    -1.6280      0.069    -23.512      0.000      -1.764      -1.492
np.maximum(Q("Carat Weight") - 2, 0)    -0.5884      0.054    -10.944      0.000      -0.694      -0.483
==============================================================================
Omnibus:                      425.364   Durbin-Watson:                   2.039
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              606.515
Skew:                           0.745   Prob(JB):                    1.98e-132
Kurtosis:                       4.009   Cond. No.                         52.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Above, we effectively have different linear relationships for our three different ranges. Let's see if it helped.

In [57]:
lm_6_predictions = lm_6.predict(df_validation)

In [58]:
mean_absolute_error(df_validation["Price_numeric"], np.exp(lm_6_predictions))

2787.961559650956

It's not as good as the best model we've found so far (that is `lm_5`), but it is significantly better than the multiplicative model (`lm_2`) which is the closest model to this one.

## Your Turn

Now that you have seen how to train linear regression models, it is your turn to see how good of a model you can build. You should mix and match all of the above techniques to build a model that performs as well as possible on the validation set. Feel free to experiment. You are likely to run into errors, try your best to figure out what the error means. It is often helpful to copy and paste the _last line_ of the error into google and see what comes up.

You should copy and paste code from above into the notebook. You can hold shift and left click on cell in order to select multiple cells. Then you can go to "Edit" in the toolbar, select "Copy Cells", move to another location in the notebook, and then select "Paste Cells Below" (or above if you so choose).

Make sure that you test your models on the validation set. You should select the model with the lowest MAE on the validation set.

I have added multiple empty cells below to encourage you to fill them, but you can always add empty cells by clicking on the "plus" icon on the toolbar. Go forth and model!

In [59]:
## Playing around with segmentation

lm_6 = smf.ols(formula='np.log(Price_numeric) ~ Q("Carat Weight") + np.maximum(Q("Carat Weight") - 1, 0) + np.maximum(Q("Carat Weight") - 2, 0)', data=df_smaller_train).fit()
lm_6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     np.log(Price_numeric)   R-squared:                       0.870
Model:                               OLS   Adj. R-squared:                  0.869
Method:                    Least Squares   F-statistic:                     9991.
Date:                   Thu, 28 Oct 2021   Prob (F-statistic):               0.00
Time:                           15:54:52   Log-Likelihood:                -233.30
No. Observations:                   4500   AIC:                             474.6
Df Residuals:                       4496   BIC:                             500.2
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                                5.7503      0.061     94.597      0.000       5.631       5.870
Q("Carat Weight")                        2.9668      0.064     46.459      0.000       2.842       3.092
np.maximum(Q("Carat Weight") - 1, 0)    -1.6280      0.069    -23.512      0.000      -1.764      -1.492
np.maximum(Q("Carat Weight") - 2, 0)    -0.5884      0.054    -10.944      0.000      -0.694      -0.483
==============================================================================
Omnibus:                      425.364   Durbin-Watson:                   2.039
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              606.515
Skew:                           0.745   Prob(JB):                    1.98e-132
Kurtosis:                       4.009   Cond. No.                         52.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [82]:
## Kitchen sink

lm_4 = smf.ols(formula='np.log(Price_numeric) ~ Cut + np.log(Q("Carat Weight")) + Color*np.log(Q("Carat Weight"))+ + Cut*np.log(Q("Carat Weight"))+Clarity + Polish + Report', data=df_smaller_train).fit()
lm_4.summary()
lm_4_predictions = lm_4.predict(df_validation)
mean_absolute_error(df_validation["Price_numeric"], np.exp(lm_4_predictions))

925.6898475144216

In [127]:
## Create your final model
lm_1 = smf.ols(formula='np.log(Price_numeric) ~ Cut + np.log(Q("Carat Weight")) + np.log(Q("Carat Weight"))*Color + np.log(Q("Carat Weight"))*Cut + np.maximum(np.log(Q("Carat Weight")) - 0.27, 0) + np.maximum(np.log(Q("Carat Weight")) - 0.27, 0)*Cut + np.maximum(np.log(Q("Carat Weight")) - .3, 0) + np.maximum(np.log(Q("Carat Weight")) - .39, 0) + np.maximum(np.log(Q("Carat Weight")) - .47, 0)+ Clarity +Polish + Symmetry + Report', data=df_smaller_train).fit()
#lm_6 = smf.ols(formula='np.log(Price_numeric) ~ Q("Carat Weight") + np.maximum(Q("Carat Weight") - 1, 0) + np.maximum(Q("Carat Weight") - 2, 0)', data=df_smaller_train).fit()
lm_1.summary()
lm_1_predictions = lm_1.predict(df_validation)
mean_absolute_error(df_validation["Price_numeric"], np.exp(lm_1_predictions))


908.0284618673976

In [62]:
print("2003.9572022239827")

2003.9572022239827


## Submitting Final Predictions

Once you have a model that you are happy with, you should predict on the test set (`df_test`), and then you should submit your predictions to Kaggle (following the instructions in your assignment sheet).

For the purpose of showing an example of how we do this, I will assume that you are selecting `lm_1` for your final model. You should obviously change the code to use the model that you would like to submit.

First, we will get the predictions on the test set. Make sure that you predict on the test set (not the validation set).

In [83]:
test_predictions = lm_4.predict(df_test)
lm_4.summary()
test_predictions
test_predictions=np.exp(test_predictions)
test_predictions

ID
6001    16362.659260
6002    44035.992609
6003     3611.031289
6004    22985.131303
6005    15045.363010
            ...     
9138     5015.245198
9139     8811.758244
9140     9648.261558
9141     7718.737032
9142     3231.640839
Length: 3142, dtype: float64

Now, we can write the predictions to a csv file. We are going to use the `.to_csv()` method. We will give it a filename to write to and a `header` parameter that will tell it what to name the predicted price column. Below we name the file "DiamondSubmission.csv", but you can choose a different name if you would like. 

In [85]:
test_predictions.to_csv("Keeley_DiamondSubmission.csv", header=["Price"])

Once you write out the file, you should see it in your file explorer sidebar (it may take a second to appear) in the folder for this class session. You can then download the file by right clicking on it and selecting download. The csv file should have two columns, the first column should be called "ID" and the second column should be called "Price". There should be 3142 predictions in your dataset (or 3143 rows with the header row).

The final step is to submit your predictions to Kaggle! Follow the instructions in the assignment.

### Bonus

In this notebook, we only trained our model on `df_smaller_train`. We used `df_validation` to select the model by measuring the performance on a held out validation set. However, after we have selected our model, there is no reason to not retrain on all of our data. Generally speaking, if a model is the best model after being trained on the smaller training set, you can feel pretty confident that it will be the best model if trained on the full training set.

The intuition here is that the more data a model has to learn, the better the model is. Once you have identified a set of variables and parameters that make up a good model, you might as well give it all of the data so that it can perform even better.

We're not going to walk through this in the notebook, but try to figure out how to train your selected model on the full training set (`df_train`) and then use this model trained on the full training set to make your predictions on the testing set.